<a href="https://colab.research.google.com/github/BrianYangatDS/gpt4-pdf-chatbot-langchain-chroma/blob/main/LangChain_QA_Panel_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain QA Panel App

This notebook shows how to make this app:

In [ ]:
!pip install langchain openai chromadb tiktoken pypdf panel


In [ ]:
!pip install unstructured

In [2]:
import os
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import panel as pn
import tempfile
from langchain.document_loaders import UnstructuredURLLoader

In [36]:
urls = [
    "https://www.ib.gov.tw/ch/home.jsp?id=42&parentpath=0,2&mcustomize=multimessages_view.jsp&dataserno=202309130002&dtable=Penalty"
]

In [37]:
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [38]:
data

[Document(page_content='選單\n\n跳到主要內容區塊\n\n金管會\n\n銀行局\n\n證期局\n\n保險局\n\n檢查局\n\n:::\r\n\t            \r\n\r\n回首頁|\r\n網站導覽|\r\nEnglish|\r\n常見問答|\r\n聯絡我們|\r\n雙語詞彙 |\r\n|\r\n|\r\n\r\n\t            \r\n\r\n字級:\r\n小\r\n中\r\n大\n\n\n\n搜尋\r\n    \r\n  搜尋\n\n關鍵字\n\n地震保險\r\n    \t    \t|\n\n強制車險\r\n    \t    \t|\n\n旅平險\r\n    \t    \t|\n\nRBC\n\n法規檢索\n\n回首頁\n\n機關介紹\r\n    \t\t      \r\n    \t\t    \t  \r\n    \t\t    \t    \r\n    \t\t    \t    \t  \r\n    \t\t    \t    \t    \r\n    \t\t    \t    \t    \t  首長簡介\r\n    \t\t    \t    \t      \r\n    \t\t    \t    \t  \r\n    \t\t    \t      \r\n    \t\t    \t    \t  \r\n    \t\t    \t    \t    \r\n    \t\t    \t    \t    \t  組織架構\r\n    \t\t    \t    \t      \r\n    \t\t    \t    \t  \r\n    \t\t    \t      \r\n    \t\t    \t    \t  \r\n    \t\t    \t    \t    \r\n    \t\t    \t    \t    \t  首長言論集\r\n    \t\t    \t    \t      \r\n    \t\t    \t    \t  \r\n    \t\t    \t      \r\n    \t\t    \t    \t  \r\n    \t\t    \t    \t    \r\n    \t\t    \t   

In [40]:
string="".join(map(str, data))
#start="".join(map(str, data)).find("英屬百慕")
end="".join(map(str, data)).rfind("公告資訊")
#string[start:end]
string[30728:end]


'新光人壽保險股份有限公司辦理高齡者投保權益業務，核有違反保險法相關規定，依保險法核處罰鍰計新臺幣120萬元整。\\n\\n2023-09-13\\n\\n一、裁罰時間：112年9月13日\\n\\n二、受處分人：新光人壽保險股份有限公司\\n\\n三、營利事業統一編號：略\\n\\n四、地址：略\\n\\n五、代表人或管理人姓名：魏○○\\n\\n六、地址：略\\n\\n七、主旨：有關本會對該公司辦理高齡者及身心障礙者投保權益作業專案檢查報告(編號：111I041)所列缺失事項，核有違反保險法相關規定，依保險法第171條之1第5項規定核處罰鍰計新臺幣(以下同)120萬元整。\\n\\n八、事實：\\n\\n(一)該公司辦理高齡者投資型保險商品適合度評估作業，業務員於招攬高齡不識字之保戶，未確實填寫高齡投保評估量表：於評估量表之項目A學歷勾選之欄位為「不識字」，而於項目F認知評估，就保戶是否有識字能力顯著低下之問項，卻勾選「無上述情形」之欄位，顯與事實不相符；且該評估結果業務員認保戶「具有辨識不利投保權益之能力且理解投保保險商品內容之適合度，其認知能力適足」，然經調閱檢視保戶銷售後承保前電訪作業之歷次錄音逐字稿，保戶於前兩次電訪未能正確回答關懷提問之問項如「第一期投資標的?」及「風險承受是屬於保守型、穩健型還是積極型?」，顯示客戶之投保權益認知不足，與高齡評估量表之評估結果未一致。\\n\\n(二)該公司雖已訂定保險業招攬及核保相關作業程序，惟高齡關懷電訪作業流於形式，且未落實執行整體保戶商品適合度作業之評估，無從確保該商品符合保戶之投保需求及適合度。\\n\\n九、理由及法令依據：\\n\\n(一)依據保險法第148條之3第2項授權訂定之「保險業招攬及核保理賠辦法」第6條第1項第6款第4目規定略以，保險業訂定其內部之業務招攬處理制度及程序，至少應包括並明定保險商品適合度政策，其內容至少應包括要保人如係投保投資型保險商品，應考量要保人之投資屬性、風險承受能力、繳交保險費之資金來源，並確定要保人已確實瞭解投資型保險之投資損益係由其自行承擔，且不得提供逾越要保人財力狀況或不合適之商品，另依同條第1項第12款第1目之2規定略以，保險業訂定其內部之招攬處理制度及程序，應包括就年齡在六十五歲以上客戶購買有保單價值準備金之保險商品者，應另指派非銷售通路之人員，於銷售保險契約後且

In [ ]:
pn.extension('texteditor', template="bootstrap", sizing_mode='stretch_width')
pn.state.template.param.update(
    main_max_width="690px",
    header_background="#F08080",
)

In [ ]:
file_input = pn.widgets.FileInput(width=300)

openaikey = pn.widgets.PasswordInput(
    value="", placeholder="Enter your OpenAI API Key here...", width=300
)
prompt = pn.widgets.TextEditor(
    value="", placeholder="Enter your questions here...", height=160, toolbar=False
)
run_button = pn.widgets.Button(name="Run!")

select_k = pn.widgets.IntSlider(
    name="Number of relevant chunks", start=1, end=5, step=1, value=2
)
select_chain_type = pn.widgets.RadioButtonGroup(
    name='Chain type',
    options=['stuff', 'map_reduce', "refine", "map_rerank"]
)

widgets = pn.Row(
    pn.Column(prompt, run_button, margin=5),
    pn.Card(
        "Chain type:",
        pn.Column(select_chain_type, select_k),
        title="Advanced settings", margin=10
    ), width=600
)

In [ ]:
def qa(file, query, chain_type, k):
    # load document
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split the documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    # select which embeddings we want to use
    embeddings = OpenAIEmbeddings()
    # create the vectorestore to use as the index
    db = Chroma.from_documents(texts, embeddings)
    # expose this index in a retriever interface
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chain to answer questions
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(), chain_type=chain_type, retriever=retriever, return_source_documents=True)
    result = qa({"query": query})
    print(result['result'])
    return result

In [ ]:
# result = qa("example.pdf", "what is the total number of AI publications?")

In [ ]:
convos = []  # store all panel objects in a list

def qa_result(_):
    os.environ["OPENAI_API_KEY"] = openaikey.value

    # save pdf file to a temp file
    if file_input.value is not None:
        file_input.save("/.cache/temp.pdf")

        prompt_text = prompt.value
        if prompt_text:
            result = qa(file="/.cache/temp.pdf", query=prompt_text, chain_type=select_chain_type.value, k=select_k.value)
            convos.extend([
                pn.Row(
                    pn.panel("\U0001F60A", width=10),
                    prompt_text,
                    width=600
                ),
                pn.Row(
                    pn.panel("\U0001F916", width=10),
                    pn.Column(
                        result["result"],
                        "Relevant source text:",
                        pn.pane.Markdown('\n--------------------------------------------------------------------\n'.join(doc.page_content for doc in result["source_documents"]))
                    )
                )
            ])
            #return convos
    return pn.Column(*convos, margin=15, width=575, min_height=400)


In [ ]:
qa_interactive = pn.panel(
    pn.bind(qa_result, run_button),
    loading_indicator=True,
)

In [ ]:
output = pn.WidgetBox('*Output will show up here:*', qa_interactive, width=630, scroll=True)

In [ ]:
# layout
pn.Column(
    pn.pane.Markdown("""
    ## \U0001F60A! Question Answering with your PDF file

    1) Upload a PDF. 2) Enter OpenAI API key. This costs $. Set up billing at [OpenAI](https://platform.openai.com/account). 3) Type a question and click "Run".

    """),
    pn.Row(file_input,openaikey),
    output,
    widgets

).servable()